# Training

In this section you can train the transformer model with Pokemon. The first section is monophony and the second with polyphony encoding.

## Train monophony encoding

In [ ]:
# Disable tensorflow warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

In [ ]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [ ]:
from mukkeBude.model import MukkeBudeTransformer
from mukkeBude.mapping import MusicMapping
import mukkeBude.utils as utils
import music21 as m21
import tensorflow as tf
import keras

# Check if GPU is found
print(tf.config.list_physical_devices('GPU'))

In [ ]:
# Create mappings
mapping = MusicMapping.create()

# optional save the mapping
# mapping.save("mapping.txt")

We use the music21 corpus for the bach training data. </br>
You can adjust the `paths` to reduce the number of training songs. </br>
</br>
See: https://web.mit.edu/music21/doc/about/referenceCorpus.html

To use custom training data use:
```python
from pathlib import Path

# Path to the input files
paths = list(Path("./dataset/Pokemon").rglob("*.midi"))
encoded_songs = utils.load_dataset_lstm(paths, 64, mapping, raw_songs=True, corpus=False)
```

In [ ]:
from pathlib import Path
paths = list(Path("../mukkeBude/songs/pokemon").rglob("*.mid"))

print(f"Found {len(paths)} songs in corpus.")

# BUG: Error - There are `music21.chord.Chord D4 G4` in the song
encoded_songs = utils.load_dataset_lstm(paths, 64, mapping, raw_songs=True, corpus=False)

Each node is represented like this:
```
c  = n60
c# = n61
d  = n62
...
```

To represent the duration of each note, we will use "`_`" for 1/16. For example, if we want to represent a C note with a duration of 1/8, we will use "`n60 _ _`". If we want to represent a C# note with a duration of 1/16, we will use onyl the letter "`n61`".

To sepperate each song there are multiple "`/`" in the dataset.

In [ ]:
# Create dataset
# Tranformer needs the dataset without xxbos and each song in one new line
utils.create_train_data(encoded_songs, "raw_train_ds_mono_videospiel.txt")
print("Dataset created")

In [ ]:
# Train model
model = MukkeBudeTransformer(mapping)
print(model)

logdir = "logs/videospiel_transformer_solo"
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model.train("raw_train_ds_mono_videospiel.txt", min_training_seq_len=32, epochs=100, tensorboard_callback=tensorboard_callback)

In [ ]:
model.save("Videospielmusik_soloMelodie_transformer")

In [ ]:
%tensorboard --logdir logs/videospiel_transformer_solo

## Train polyphony encoding

In [ ]:
# Disable tensorflow warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

In [ ]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [ ]:
from mukkeBude.model import MukkeBudeTransformer
from mukkeBude.mapping import MusicMapping
import mukkeBude.utils as utils
import music21 as m21
import tensorflow as tf
import keras

# Check if GPU is found
print(tf.config.list_physical_devices('GPU'))

In [ ]:
# Create mappings
mapping = MusicMapping.create()

# optional save the mapping
# mapping.save("mapping.txt")

We use the music21 corpus for the bach training data. </br>
You can adjust the `paths` to reduce the number of training songs. </br>
</br>
See: https://web.mit.edu/music21/doc/about/referenceCorpus.html

To use custom training data use:
```python
from pathlib import Path

# Path to the input files
paths = list(Path("./dataset/Pokemon").rglob("*.midi"))
```

In [ ]:
from pathlib import Path
paths = list(Path("../mukkeBude/songs/pokemon").rglob("*.mid"))

print(f"Found {len(paths)} songs in corpus.")

encoded_songs = []
for path in paths:
    song = utils.read_single(path)
    encoded_song = utils.to_polyphonic_encoding(song, mapping)
    encoded_songs.append(mapping.textify(encoded_song))

print(f"Songs encoded: {len(encoded_songs)}")

Each node is represented like this:
```
c  = n60
c# = n61
d  = n62
...
```

To represent the duration of each note, we will use "`d1, d2, d3, ...`". For example, if we want to represent a C note with a duration of 1/8, we will use "`n60 d2`". If we want to represent a C# note with a duration of 1/16, we will use "`n61 d1`".

To sepperate the notes which are played at the same time we will use "`xxsep`". For example, if we want to represent a C note with a duration of 1/16 and a C# note with a duration of 1/16 played at the same time, we will use "`n60 d1 n61 d1 xxsep`".

To sepperate each song there is "`xxbos`" in the dataset.

In [ ]:
# Create dataset
# Tranformer needs the dataset without xxbos and each song in one new line
utils.create_train_data(encoded_songs, "raw_train_ds_videospiel.txt")
print("Dataset created")

In [ ]:
# Train model
model = MukkeBudeTransformer(mapping)
print(model)

logdir = "logs/videospiel_transformer_polyphonie"
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model.train("raw_train_ds_videospiel.txt", min_training_seq_len=32, epochs=100, tensorboard_callback=tensorboard_callback)

In [ ]:
model.save("Videospielmusik_polyphonie_transformer")

In [ ]:
%tensorboard --logdir logs/videospiel_transformer_polyphonie

# Generate music

In this section you can generate music with a pre trained transformer model. The first section is monophony and the second with polyphony encoding.

## Generate monophony

In [ ]:
# Disable tensorflow warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

In [ ]:
from mukkeBude.mapping import MusicMapping
from mukkeBude.model import MukkeBudeTransformer
import mukkeBude.utils as utils
import tensorflow as tf
from pathlib import Path

from mukkeBude.mapping import SPECIAL_TOKS
from mukkeBude.mapping import REST
from mukkeBude.mapping import WAIT_LSTM

# Check if GPU is found
print(tf.config.list_physical_devices('GPU'))

In [ ]:
# Create mappings
mapping = MusicMapping.create()

# optional save the mapping
# mapping.save("mapping.txt")

You have to know the trainings data and the same trainings parameter.

In [ ]:
model = MukkeBudeTransformer.load(mapping, "Videospielmusik_soloMelodie_transformer", "raw_train_ds_mono_videospiel.txt", min_training_seq_len=32)

In [ ]:
# Create song
# TODO: Right start seed
# generated_song = model.generate("n60 _ _ _ n55 _ _ _ n52 _ _ _ n48 _ n47 _ n60 _ _ _ n60", max_length=500)

# Remove REST and WAIT_LSTM from SPECIAL_TOKS
# They should not be removed from the generated song
special_tokens = SPECIAL_TOKS.copy()
special_tokens.remove(REST)
special_tokens.remove(WAIT_LSTM)

generated_song = " ".join(utils.replace_special_tokens(generated_song.split(), WAIT_LSTM, special_tokens))

In [ ]:
new_song = utils.decode_songs_old(generated_song)
print(generated_song)

In [ ]:
path = Path("generated_song_pokemon_trans_mono.midi")
utils.write_midi(new_song, path)

## Generate polyphony

In [ ]:
# Disable tensorflow warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

In [ ]:
from mukkeBude.mapping import MusicMapping
import mukkeBude.utils as utils
import tensorflow as tf
import numpy as np
from pathlib import Path

from mukkeBude.mapping import SPECIAL_TOKS
from mukkeBude.mapping import SEP
from mukkeBude.mapping import BOS

# Check if GPU is found
print(tf.config.list_physical_devices('GPU'))

In [ ]:
# Create mappings
mapping = MusicMapping.create()

# optional save the mapping
# mapping.save("mapping.txt")

You have to know the trainings data and the same trainings parameter.

In [ ]:
model = MukkeBudeTransformer.load(mapping, "Videospielmusik_polyphonie_transformer", "raw_train_ds_videospiel.txt", min_training_seq_len=32)

In [ ]:
# Create song
generated_song = model.generate("n46 d4 xxsep d4 n53 d2 n50 d2 xxsep d2 n62 d1 n58 d1 xxsep d2 n53 d2 n53 d2 n50 d2 xxsep d4 n46 d4 xxsep d4 n55 d2 n51 d2 xxsep d2 n63 d1 n58 d1 xxsep d2", max_length=1000, probability=0.8)

# Remove REST and WAIT_LSTM from SPECIAL_TOKS
# They should not be removed from the generated song

special_tokens = SPECIAL_TOKS.copy()
special_tokens.remove(SEP)
special_tokens.remove(BOS)

generated_song = " ".join(utils.replace_special_tokens(generated_song.split(), "d1", special_tokens))
print(generated_song)

In [ ]:
# Convert to music21
new_song_ints = mapping.numericalize(generated_song.split(" "))
new_song_ints = np.array(new_song_ints)

new_song = utils.from_polyphonic_encoding(new_song_ints, mapping, bpm=140)

path = Path("generated_song_pokemon_trans_poly.midi")
utils.write_midi(new_song, path)
